# SyGMA processing for metabolites

Info: Jupyter notebook made by LF and Pryia on April 2018

Description: Process a list of structures (SMILES) and generate Phase I and II metabolites

In [3]:
import pandas as pd
import numpy as np
import sygma
from rdkit import Chem

In [31]:
#Open dataframes
df = pd.read_csv("Saliva_InChI.txt",  sep='\t', header=0, encoding='utf-8')
df2 = pd.read_csv("drugbank_approved_structure_links.csv",  sep=',', header=0, encoding='latin-1')
df3 = pd.read_csv("drugbank_illicit_structure_links.csv",  sep=',', header=0, encoding='latin-1')
df4 = pd.read_csv("drugbank_nutraceutical_structure_links.csv",  sep=',', header=0, encoding='latin-1')
df5 = pd.read_csv("FOODB_compounds_3.txt",  sep='\t', header=0, encoding='utf-8')
df6 = pd.read_csv("Exposome_explorer_biomarkers.csv",  sep=',', header=0)
df7 = pd.read_csv("toxins_edited_v2.txt",  sep='\t', header=0, encoding='latin-1')
df8 = pd.read_csv("PTID_Pesticide_3D-Plant2Cells_compound.txt",  sep='\t', header=0, encoding='latin-1')
df6.head(5)

,ID,Name,Synonyms,Level,Description,CAS Number,PubChem ID,ChEBI ID,FooDB ID,HMDB ID,SMILES,Formula,InChI,InChIKey,Average mass,Mono. mass
0,1,Polyphenols,NaN,Combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,Isoflavones,NaN,Combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Genistein,NaN,Single,NaN,446-72-0,5280961.0,28088.0,FDB011828,HMDB03217,OC1=CC=C(C=C1)C1=COC2=CC(O)=CC(O)=C2C1=O,C15H10O5,InChI=1S/C15H10O5/c16-9-3-1-8(2-4-9)11-7-20-13...,TZBJGXHYKVUXJN-UHFFFAOYSA-N,270.2400,270.052823
3,4,Daidzein,NaN,Single,NaN,486-66-8,5281708.0,28197.0,FDB002608,HMDB03312,OC1=CC=C(C=C1)C1=COC2=C(C=CC(O)=C2)C1=O,C15H10O4,InChI=1S/C15H10O4/c16-10-3-1-9(2-4-10)13-8-19-...,ZQSIJRDFPHDXIC-UHFFFAOYSA-N,254.2410,254.057909
4,10,Equol,NaN,Single,NaN,531-95-3,91469.0,34741.0,FDB021824,HMDB02209,OC1=CC=C(C=C1)[C@H]1COC2=C(C1)C=CC(O)=C2,C15H14O3,InChI=1S/C15H14O3/c16-13-4-1-10(2-5-13)12-7-11...,ADFCQWZHKCXPAJ-GFCCVEGCSA-N,242.2699,242.094294


In [32]:
#Keep just a column per dataset
df = df[['SMILES']]
df2 = df2[['SMILES']]
df3 = df3[['SMILES']]
df4 = df4[['SMILES']]
df5 = df5[['SMILES']]
df6 = df6[['SMILES']]
df7 = df7[['SMILES']]
df8 = df8[['SMILES']]

In [57]:
#Concatenate tables
frames = [df, df2, df3, df4, df5, df6, df7, df8]
Table = pd.concat(frames)


AttributeError: module 'pandas' has no attribute 'append'

In [56]:
Table.head(25)

,DrugBank ID,HMDB ID,PTID_num,SMILES,T3DB ID,ID
0,DB00006,NaN,NaN,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,NaN,NaN
1,DB00014,NaN,NaN,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,NaN,NaN
2,DB00027,NaN,NaN,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,NaN,DB00027_DB00191_DB00116_HMDB03217_T3D0802_1043
3,DB00035,NaN,NaN,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,NaN,DB00035_DB00230_DB00117_HMDB03312_T3D1641_1452
4,DB00050,NaN,NaN,CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H...,NaN,DB00050_DB00237_DB00118_HMDB02209_T3D0671_509
5,DB00067,NaN,NaN,NaN,NaN,DB00067_DB00241_DB00119_HMDB04629_T3D1450_909
6,DB00080,NaN,NaN,CCCCCCCCCC(=O)N[C@@H](CC1=CNC2=C1C=CC=C2)C(=O)...,NaN,DB00080_DB00306_DB00120_HMDB05781_T3D3961_1523
7,DB00091,NaN,NaN,CC[C@@H]1NC(=O)[C@H]([C@H](O)[C@H](C)C\C=C\C)N...,NaN,DB00091_DB00318_DB00121_HMDB05760_T3D3652_1000
8,DB00093,NaN,NaN,NCCCC[C@H](NC(=O)[C@@H]1CCCN1C(=O)[C@@H]1CSSC[...,NaN,DB00093_DB00327_DB00122_HMDB05897_T3D1780_1422
9,DB00104,NaN,NaN,[H][C@]1(NC(=O)[C@H](CCCCN)NC(=O)[C@@H](CC2=CN...,NaN,DB00104_DB00349_DB00123_HMDB05794_T3D4476_1500


In [37]:
Table.shape

(16966, 5)

In [38]:
len(Table.SMILES.unique())

14735

In [39]:
#Remove empty rows
Table_clean = Table[Table.SMILES.str.contains('nan') == False]

# Remove steoreochemistry from SMILES
Table_clean['SMILES'].replace(regex=True,inplace=True,to_replace=r'@@',value=r'')
Table_clean['SMILES'].replace(regex=True,inplace=True,to_replace=r'@',value=r'')
Table_clean['SMILES'].replace(regex=True,inplace=True,to_replace=r'/',value=r'')
Table_clean['SMILES'].replace(regex=True,inplace=True,to_replace=r'\\',value=r'')

#Filter SMILES and Formula lenght
Table_clean = Table_clean[Table_clean['SMILES'].map(len) > 8]
Table_clean = Table_clean[Table_clean['SMILES'].map(len) < 125]

#Sort by InChI lenght
Table_clean.sort_values('SMILES',inplace=True, ascending=False)
Table_clean.index = Table_clean['SMILES'].str.len()
Table_clean = Table_clean.sort_index(ascending=False).reset_index(drop=True)
Table_clean.shape

/home/labshare/.local/lib/python3.6/site-packages/pandas/core/generic.py:4619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


(12748, 5)

In [40]:
#Write the file out
Table_clean.to_csv('Combined_DB_DrugDB_FoodDB_Pesticide_Exposome_Sygma_SMILES.tsv', sep = '\t', index = False)

In [259]:
#Make full list from SMILES column
List_smiles = []
List_smiles = Table_clean['SMILES'].tolist()
List_smiles

#Remove duplicates
List_smiles_unique = []
for item in List_smiles:
    if item not in List_smiles_unique:
        List_smiles_unique.append(item)
        
print(len(List_smiles_unique))

11288


## Running SyGMa

Each step in a scenario lists the ruleset
and the number of reaction cycles to be applied

In [280]:
# Define SyGMA scenario. 
scenario = sygma.Scenario([
    [sygma.ruleset['phase1'], 1],
    [sygma.ruleset['phase2'], 1]])

In [281]:
# Create a list of RDkit object from smiles
list_mol = [Chem.MolFromSmiles(x) for x in List_smiles_unique]
list_mol

 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 ...]

In [282]:
# Remove None
list_mol_clean = [x for x in list_mol if x!=None]

In [283]:
print(len(list_mol))
print(len(list_mol_clean))

11288
10110


In [284]:
df = pd.DataFrame(data=None)
df2 = pd.DataFrame(data=None)
for x in list_mol_clean:
    print(x)
    metabolic_tree = scenario.run(x)
    metabolic_tree.calc_scores()
    metabolites = metabolic_tree.to_smiles()
    df = pd.DataFrame(metabolites[1:],columns=metabolites[0])
    df['parent'] = (metabolites[0][0])
    df.columns.values[0] = 'metabolite'
    df.columns.values[1] = 'score'
    #Keep only top20
    df2 = df2.append(df[:20], ignore_index=True)

ZeroDivisionError: float division by zero

In [286]:
df2.shape

(2869, 3)

In [290]:
#Number of unique parent
len(df2.parent.unique())

142

In [288]:
df2.head(5)

,metabolite,score,parent
0,CCCCCCCC(O)CC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C)C...,0.271,CCCCCCCC(O)CC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C)C...
1,CCCCCCCC(O)CC(=O)S(=O)CCNC(=O)CCNC(=O)C(O)C(C)...,0.237,CCCCCCCC(O)CC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C)C...
2,CCCCCCCC(O)CC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C)CO,0.203,CCCCCCCC(O)CC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C)C...
3,Nc1ncnc2c1ncn2C1OC(COP(=O)(O)OP(=O)(O)O)C(OP(=...,0.203,CCCCCCCC(O)CC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C)C...
4,CCCCCCCC(O)CC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C)C...,0.203,CCCCCCCC(O)CC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C)C...


In [289]:
#Write the file out
df2.to_csv('Combined_DB_DrugDB_FoodDB_Pesticide_Exposome_Sygma_1_1_metabolites.tsv', sep = '\t', index = False)